In [ ]:
%pip install opencv-python
%matplotlib inline

## Idea:
1. Crop lines from the page
- get their transcription using another model.
2. Give the lines to the model I'm fine tunning.
3. Build the text from lines back into the whole page.

In [ ]:
import cv2
import os
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:
# image_folder = "text-lvl1"
# image_path = f"full-datasets/{image_folder}"

image_path = "C:/Users/dhlabadmin/Desktop/m-test/Fine-Tuning-TrOCR/datasets/dataset_6470048/"

In [ ]:
def do_segmentation(im_path, im_name):
    ouput_path = f"segmentation-ouput/{im_name}/"
    os.makedirs(os.path.dirname(ouput_path), exist_ok=True)
    # raw_img = cv2.imread(im_path)

    # target_heigh = 1024
    # scale = target_heigh/raw_img.shape[0]
    # new_width = int(raw_img.shape[1]*scale)
    # img = cv2.resize(raw_img, (new_width, target_heigh), interpolation=cv2.INTER_AREA)

    # cv2.imwrite("img_resized.jpg", img)

    img = cv2.imread(im_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # plt.imshow(img_gray)
    # plt.show()

    # separate foreground and background
    ret, bin_img = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY_INV)
    # remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (150,1))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_DILATE, kernel)

    # plt.imshow(bin_img)
    # plt.show()

    bboxes = []
    bboxes_img = img.copy()
    contours = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(bboxes_img, (x, y), (x+w, y+h), (0,0,255), 1)
        bboxes.append((x,y,w,h))
    
    # plt.imshow(bboxes_img)
    # plt.show()

    crops = []
    filtered = []
    for j in range(len(bboxes)):
        (x,y,w,h) = bboxes[j]
        crop = img[y-10:y+h+10, x-10:x+w+10]
        if crop.size == 0:
            continue
        if crop.size > 500: # <-- added
            crops.append(crop)
            filtered.append((x,y,w,h))
    
    # aver_width = sum(c.shape[1] for c in crops) / len(crops)
    # aver_height = sum(c.shape[0] for c in crops) / len(crops)
    
    # print(aver_width)
    # print(aver_height)

    for j in range(len(crops)):
        c = crops[j]
        # if c.shape[1] > 500:
        filename = os.path.join(ouput_path, f"S_{j}.jpg")
        cv2.imwrite(filename, c)

In [ ]:
for file_name in tqdm(os.listdir(image_path)):

    if file_name.endswith((".jpg", ".png", ".tif", "jpeg")):
        image_name = os.path.join(image_path, file_name)

        try:
            do_segmentation(image_name, file_name)

        except Exception as e:
            print(f"Error on processing {file_name}: {e}")